# AMAZON WEB SCRAPER


In [18]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
import re

def get_url(search_term):
    #Generate a url for search term
    template ='https://www.amazon.in/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ','+')
    
    url= template.format(search_term)
    
    url += '&page{}'
    
    return url



def extract_record(item):
    
    #product description
    
    atag=item.h2.a
    tag=atag.text
    product_name = atag.text.split('(')[0].strip()
    RAM = ''.join(re.findall("([0-9]*GB) RAM", tag))
    storage = ''.join(re.findall("([0-9]*GB) Storage", tag))
    battery= ''.join(re.findall("([0-9]*) mAh", tag))
    url='https://www.amazon.in'+ atag.get('href')
        
    
    #product price 
    try:
        price= item.find('span', 'a-price-whole').text
        
    except AttributeError:
        price = None
        
    #product rating and reviews
    try:
        rating_string=item.i.text.strip()
        rating=''.join(re.findall("([0-9].*) out", rating_string))
        review=item.find('span','a-size-base').text.split(",")
        review="".join(review)
        if review.isdigit():
            review_count= review
        else:
            review_count=0
        
    except AttributeError:
        rating= None
        review_count= 0

    result= (product_name, RAM, storage, battery, price, rating, review_count, url) 
    
    return result


def main(search_term):
    driver = webdriver.Chrome("C:\Program Files\chromedriver") 
    records=[]
    url= get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup( driver.page_source, 'html.parser')
        results= soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    
    with open('Mobiles over 30000.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Product Name','RAM','Storage','Battery(mAh)','Price','Rating','Review Count','URL'])
        writer.writerows(records)
        

In [19]:
main('Mobiles over 30000')